In [ ]:
%pip install tensorrt-cu12-libs onnx_asr[gpu,hub]

In [ ]:
import tensorrt_libs  # noqa: F401

import onnx_asr
from onnx_asr.utils import read_wav_files

model = onnx_asr.load_model("gigaam-v3-ctc")

In [ ]:
for k in [1, 4, 16]:
    print(f"Batch size: {k}")

    data = ["test.wav"] * k
    waveform = read_wav_files(data, 16_000)
    resampled = model.resampler(*waveform)
    preprocessed = model.asr._preprocessor(*resampled)
    encoded = model.asr._encode(*preprocessed)

    t1 = %timeit -o -q model.resampler(*waveform)
    t2 = %timeit -o -q model.asr._preprocessor(*resampled)
    t3 = %timeit -o -q model.asr._encode(*preprocessed)
    t4 = %timeit -o -q list(model.asr._decoding(*encoded))
    t5 = %timeit -o -q model.recognize(data)

    for res in [t1, t2, t3, t4, t5]:
        res.timings = [t / k for t in res.timings]

    print(f"resampler: {t1}")
    print(f"preprocessor: {t2}")
    print(f"encoder: {t3}")
    print(f"decoding: {t4}")
    print(f"full recognize: {t5}")
    print()